In [7]:
from dotenv import load_dotenv
import os
import streamlit as st
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from langchain.callbacks import get_openai_callback
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.embeddings import GPT4AllEmbeddings
import glob
from tqdm import tqdm
import pickle
import json
import numpy as np

# load qa dataset
# Reading JSON data
json_filename = "QA_dataset.json"
with open(json_filename, 'r') as file:
    data_loaded = json.load(file)

correctness = []
for item in tqdm(data_loaded):
    
    try:
        pdf = item["filename"]
        ques= [item["question_1"] , item["question_2"], item["question_3"]]
        anss = [item["answer_1"], item["answer_2"], item["answer_3"]]
    except:
        print("Key error, please check. ", item)
        continue
    
    pdf_reader = PdfReader(pdf)
    text = ""
    for page in pdf_reader.pages:
        text += page.extract_text()


    os.environ["OPENAI_API_KEY"] = # ENTER_HERE

    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )  

    chunks = text_splitter.split_text(text)

    # embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
    embeddings = GPT4AllEmbeddings()
    save_faiss_index = False
    index_filename = "faiss_index.gpt4all"
    if save_faiss_index:
        if not os.path.exists(index_filename):
            knowledge_base = FAISS.from_texts(chunks, embeddings)
            knowledge_base.save_local(index_filename)
        else:
            knowledge_base = FAISS.load_local(index_filename, embeddings, allow_dangerous_deserialization=True)
    else:
        knowledge_base = FAISS.from_texts(chunks, embeddings)
        
    for query, answer in zip(ques, anss):
        docs = knowledge_base.similarity_search(query)

        llm = OpenAI()
        chain = load_qa_chain(llm, chain_type="stuff")
        response = chain.run(input_documents=docs, question=query)

        print(f"Query: {query}")
        print(f"Ans: {answer}")
        print(f"Res: {response}")
        
        verifier_prompt = """## Role: Answer verifier

## Goal
You can judge whether the answer is correct or not. 

## Rule
- If the key information predicted answer is same as the ground truth answer, then the answer is correct.


## Output format
{
"reason": "fill the reason why the predicted answer is wrong (False) or correct (True).", 
"answer": True or False
}

""" + f"""

The question is: {query}
Ground truth is: {answer}
Predicted answer is: {response}
"""
        # try:
        verified_output = llm.invoke(verifier_prompt)
        print("verified_output: ", verified_output)
        verified_bool = verified_output.split('"answer": ')[-1]
        if "True" in verified_bool:
            verified_bool = 1
        else:
            verified_bool = 0
        # except:
        #     verified_bool = 0
        
        print(f"Correct or not: {verified_bool}")
        correctness.append(verified_bool)
        
print(correctness)
print(np.mean(correctness))

  0%|          | 0/15 [00:00<?, ?it/s]

Query: What was Amazon's total revenue in 2023?
Ans: $575B
Res:  Amazon's total revenue in 2023 was $575 billion.
verified_output:  
{
"reason": "The predicted answer is the same as the ground truth answer, which is $575B. Therefore, the answer is correct.", 
"answer": True
}
Correct or not: 1
Query: By what percentage did Amazon's AWS revenue increase in 2023?
Ans: 13%
Res:  13%
verified_output:  
{
"reason": "The predicted answer is same as the ground truth answer.", 
"answer": True
}
Correct or not: 1
Query: How much did Amazon's operating income improve in 2023 from the previous year?
Ans: 201%
Res:  Operating income in 2023 improved 201% Y oY from $12.2B to $36.9B. This is a dramatic improvement of $24.7B.


  7%|▋         | 1/15 [00:38<09:03, 38.82s/it]

verified_output:  {
"reason": "The answer is correct because it accurately states the percentage improvement and the specific numbers for both the previous year and 2023. The language used is also consistent with the question, making it a valid answer.", 
"answer": True
}
Correct or not: 1
Key error, please check.  {'question_1': 'When was Huawei founded?', 'question_2': 'How much revenue did Huawei generate in 2023?', 'question_3': 'How many employees does Huawei have in 2023?', 'filename': './docs\\annual_report_2023_en.pdf', 'first_5_page_texts': 'Huawei Investment & Holding Co., Ltd.\n2023 ANNUAL REPORTWho is Huawei?\nFounded in 1987, Huawei is a leading \nglobal provider of information and \ncommunications technology (ICT) \ninfrastructure and smart devices. We \nhave approximately 207,000 employees \nand we operate in over 170 countries \nand regions, serving more than three \nbillion people around the world. We are \ncommitted to bringing digital to every \nperson, home and orga

Created a chunk of size 2612, which is longer than the specified 1000
Created a chunk of size 1801, which is longer than the specified 1000


...

verified_output:  
{
"reason": "The predicted answer is the same as the ground truth, so the answer is correct.", 
"answer": True
}
Correct or not: 1


Created a chunk of size 2512, which is longer than the specified 1000
Created a chunk of size 1280, which is longer than the specified 1000
Created a chunk of size 1591, which is longer than the specified 1000
Created a chunk of size 1031, which is longer than the specified 1000
Created a chunk of size 1256, which is longer than the specified 1000
Created a chunk of size 1077, which is longer than the specified 1000


Query: When did SoftBank Group Corp. start preparing its consolidated financial statements in accordance with the International Financial Reporting Standards (IFRS)?
Ans: From the three-month period ended June 30, 2013
Res: 

The Company has prepared the consolidated financial statements in accordance with the International Financial Reporting Standards (IFRS) from the three-month period ended June 30, 2013.
verified_output:  
{
"reason": "The predicted answer is the same as the ground truth answer, so it is correct.", 
"answer": True
}
Correct or not: 1
Query: What is SoftBank Group Corp.'s (SBG) mission as stated in the 2023 annual report?
Ans: Information Revolution—Happiness for everyone
Res:  The 2023 annual report does not state a specific mission for SBG.
verified_output:  {
"reason": "The predicted answer does not match the ground truth answer.", 
"answer": False
}
Correct or not: 0
Query: What historical figure and organization does SBG draw inspiration from, as mentioned in t

100%|██████████| 15/15 [14:17<00:00, 57.19s/it]

verified_output:  {
"reason": "The predicted answer is the same as the ground truth.", 
"answer": True
}

Correct or not: 1
[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1]
0.75
